#### LeetCode - SQL - #1757

Refer [1757. Recyclable and Low Fat Products](https://leetcode.com/problems/recyclable-and-low-fat-products/description/)

Write a solution to find the ids of products that are both low fat and recyclable.
1. low_fats is an ENUM (category) of type ('Y', 'N') where 'Y' means this product is low fat and 'N' means it is not.
2. recyclable is an ENUM (category) of types ('Y', 'N') where 'Y' means this product is recyclable and 'N' means it is not.

Return the result table in  **any order**.

In [0]:
data = [['0', 'Y', 'N'], ['1', 'Y', 'Y'], ['2', 'N', 'Y'], ['3', 'Y', 'Y'], ['4', 'N', 'N']]

columns = ['product_id', 'low_fats', 'recyclable']
schema = {'product_id':'int64', 'low_fats':'category', 'recyclable':'category'}

In [0]:
import pandas

pandas_dataframe = pandas.DataFrame(data=data, columns=columns).astype(schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
spark_dataframe_using_pandas = spark_context.createDataFrame(pandas_dataframe)
spark_dataframe_using_pandas.printSchema()
spark_dataframe_using_pandas.show()

root
 |-- product_id: long (nullable = true)
 |-- low_fats: string (nullable = true)
 |-- recyclable: string (nullable = true)

+----------+--------+----------+
|product_id|low_fats|recyclable|
+----------+--------+----------+
|         0|       Y|         N|
|         1|       Y|         Y|
|         2|       N|         Y|
|         3|       Y|         Y|
|         4|       N|         N|
+----------+--------+----------+



In [0]:
spark_dataframe_using_pandas.select("product_id").where("low_fats = 'Y'").where("recyclable = 'Y'").show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+



In [0]:
spark_dataframe_using_pandas.select("product_id").where("low_fats = 'Y' AND recyclable = 'Y'").show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+



In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col

# Spark DataFrame directly using data and schema
data_schema = StructType([
    StructField("product_id", StringType(), False),
    StructField("low_fats", StringType(), True),
    StructField("recyclable", StringType(), True),
])

spark_dataframe = spark_context.createDataFrame(data=data, schema=data_schema)
spark_dataframe.printSchema()

# Note that the product_id is of String type
# Convert String type to Integer type

## Using withColumn function
spark_dataframe = spark_dataframe.withColumn("product_id", spark_dataframe["product_id"].cast(IntegerType()))
# spark_dataframe = spark_dataframe.withColumn("product_id", col("product_id").cast(IntegerType()))
# spark_dataframe = spark_dataframe.withColumn("product_id", col("product_id").cast("int"))
# spark_dataframe = spark_dataframe.withColumn("product_id", col("product_id").cast("integer"))

## Using select function
# spark_dataframe = spark_dataframe.select(col("product_id").cast("int"), col("low_fats"), col("recyclable"))
# spark_dataframe = spark_dataframe.selectExpr("CAST(product_id AS int) AS product_id", "low_fats", "recyclable")
# spark_dataframe = spark_dataframe.selectExpr("INT(product_id ) AS product_id", "low_fats", "recyclable")

## Using SQL query
# spark_dataframe.createOrReplaceTempView("ProductsTempView")
# spark_dataframe = spark_context.sql("SELECT CAST(product_id AS int) AS product_id, low_fats, recyclable FROM ProductsTempView")

spark_dataframe.printSchema()
spark_dataframe.show()

root
 |-- product_id: string (nullable = false)
 |-- low_fats: string (nullable = true)
 |-- recyclable: string (nullable = true)

root
 |-- product_id: integer (nullable = true)
 |-- low_fats: string (nullable = true)
 |-- recyclable: string (nullable = true)

+----------+--------+----------+
|product_id|low_fats|recyclable|
+----------+--------+----------+
|         0|       Y|         N|
|         1|       Y|         Y|
|         2|       N|         Y|
|         3|       Y|         Y|
|         4|       N|         N|
+----------+--------+----------+

